In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from pathlib import Path

# ================================
# Define helper functions
# ================================

# Convert emissions to metric tonnes
def convert_to_tonnes(row):
    emis_value = row.get('ann_value') if 'ann_value' in row else 0
    emis_units = row.get('emissions uom') if 'emissions uom' in row else 'TON'
    
    if pd.isna(emis_value) or emis_value == '':
        return 0
        
    emis_value = float(emis_value)
    
    if emis_units == 'LB':
        return emis_value * 0.000453592  # Convert pounds to metric tonnes
    elif emis_units == 'TON':
        return emis_value * 0.90718474  # Convert short tons to metric tonnes
    return emis_value  # Already in metric tonnes

# Categorize pollutants
def categorize_pollutant(row):
    pollutant = str(row.get('poll', '')).upper()
    
    # Attempt to get pollutant description if available
    pollutant_desc = ''
    if 'pollutant desc' in row:
        pollutant_desc = str(row['pollutant desc']).upper()
    
    if pollutant == 'VOC' or 'VOLATILE ORGANIC' in pollutant_desc:
        return 'VOC'
    elif pollutant in ['NOX', 'NO', 'NO2'] or ('NITROGEN' in pollutant_desc and 'OXIDE' in pollutant_desc):
        return 'NOx'
    elif pollutant == 'NH3' or 'AMMONIA' in pollutant_desc:
        return 'NH3'
    elif pollutant in ['SO2', 'SO4'] or 'SULFUR' in pollutant_desc:
        return 'SOx'
    elif 'PM25' in pollutant or 'PM2.5' in pollutant_desc or 'PM2_5' in pollutant:
        return 'PM2_5'
    elif pollutant == 'CO2':
        return 'CO2'
    return 'Other'

# ================================
# Process the FF10_POINT file for InMAP compatibility
# ================================

# Path to your specific FF10_POINT format file
file_path = "../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv"

print(f"Reading data from {file_path}...")

# Create output directory if it doesn't exist
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Count number of header lines to skip
with open(file_path, 'r') as f:
    header_lines = 0
    for line in f:
        if line.startswith('#'):
            header_lines += 1
        else:
            break

# Read the FF10_POINT file, skipping header comments
df = pd.read_csv(file_path, skiprows=header_lines, low_memory=False)

print(f"Data loaded. Shape: {df.shape}")

# ================================
# Load CEMS facilities for filtering
# ================================

# Load CEMS data to get valid facility IDs
print("Loading CEMS facilities data...")
emissions_cems = pd.read_csv('../data/2023_annual_emissions_CEMS.csv')
cems_plants = set(emissions_cems['Facility ID'].unique())  # Convert to set for faster lookups
print(f"Loaded {len(cems_plants)} unique CEMS facility IDs")

# ================================
# Process emissions data
# ================================

# Initialize dictionaries to store emissions by location
location_data = {}

# Process each row of the FF10_POINT file
for idx, row in df.iterrows():
    try:
        # Get ORIS facility code
        oris_code = row['oris_facility_code'] if 'oris_facility_code' in row else None
        
        # Clean and validate ORIS code
        if oris_code is None or pd.isna(oris_code) or str(oris_code).strip() == '':
            continue
            
        # Try to convert to integer, skipping if not possible
        try:
            oris_code_int = int(str(oris_code).strip())
            if oris_code_int not in cems_plants:
                continue
        except (ValueError, TypeError):
            continue
            
        # Get coordinates
        lon = row['longitude'] if 'longitude' in row else None
        lat = row['latitude'] if 'latitude' in row else None
        
        if lon is None or lat is None or pd.isna(lon) or pd.isna(lat):
            continue  # Skip records without valid coordinates
        
        # Create a location key for aggregating emissions
        loc_key = (float(lon), float(lat), str(oris_code_int))  # Include ORIS code in the key
        
        # Get stack parameters
        h = row['stkhgt'] if 'stkhgt' in row else ''
        if h != '' and not pd.isna(h):
            # Convert feet to meters
            h_val = float(h) * 0.3048
        else:
            h_val = 0
            
        d = row['stkdiam'] if 'stkdiam' in row else ''
        if d != '' and not pd.isna(d):
            # Convert feet to meters
            d_val = float(d) * 0.3048
        else:
            d_val = 0
            
        t = row['stktemp'] if 'stktemp' in row else ''
        if t != '' and not pd.isna(t):
            # Convert F to K
            t_val = (float(t) - 32) * 5.0/9.0 + 273.15
        else:
            t_val = 0
            
        v = row['stkvel'] if 'stkvel' in row else ''
        if v != '' and not pd.isna(v):
            # Convert ft/s to m/s
            v_val = float(v) * 0.3048
        else:
            v_val = 0
        
        # Get NAICS code
        naics = row['naics'] if 'naics' in row else None
        naics_str = str(naics) if naics is not None and not pd.isna(naics) else ''
        
        # If this is the first emission value for this location, initialize
        if loc_key not in location_data:
            location_data[loc_key] = {
                'VOC': 0, 'NOx': 0, 'NH3': 0, 'SOx': 0, 'PM2_5': 0, 'CO2': 0,
                'height': h_val, 'diam': d_val, 'temp': t_val, 'velocity': v_val,
                'naics': naics_str, 'oris_code': oris_code_int
            }
        
        # Convert emissions to metric tonnes
        emissions_tonnes = convert_to_tonnes(row)
        
        # Get pollutant category
        pollutant_category = categorize_pollutant(row)
        
        # Add emissions to the appropriate category
        if pollutant_category in location_data[loc_key]:
            location_data[loc_key][pollutant_category] += emissions_tonnes
            
    except Exception as e:
        if idx % 5000 == 0:  # Only print errors every 5000 rows to avoid flooding output
            print(f"Error processing row {idx}: {e}")
        continue

# Convert the aggregated location data into lists for the GeoDataFrame
coords = []
VOC, NOx, NH3, SOx, PM2_5, CO2 = [], [], [], [], [], []
height, diam, temp, velocity = [], [], [], []
naics_codes = []
oris_codes = []

for loc, data in location_data.items():
    coords.append(Point(loc[0], loc[1]))
    VOC.append(data['VOC'])
    NOx.append(data['NOx'])
    NH3.append(data['NH3'])
    SOx.append(data['SOx'])
    PM2_5.append(data['PM2_5'])
    CO2.append(data['CO2'])
    height.append(data['height'])
    diam.append(data['diam'])
    temp.append(data['temp'])
    velocity.append(data['velocity'])
    naics_codes.append(data['naics'])
    oris_codes.append(data['oris_code'])

print(f"Processed {len(coords)} unique emission locations that match CEMS facility IDs")

# Create the emissions GeoDataFrame
data_dict = {
    "VOC": VOC, 
    "NOx": NOx, 
    "NH3": NH3, 
    "SOx": SOx, 
    "PM2_5": PM2_5,
    "CO2": CO2,
    "height": height, 
    "diam": diam, 
    "temp": temp, 
    "velocity": velocity,
    "naics_code": naics_codes,
    "oris_facility_code": oris_codes
}

emis = gpd.GeoDataFrame(data_dict, geometry=coords, crs='epsg:4269')

# Filter out any rows with all zeros for InMAP-relevant emissions
emis = emis[(emis['VOC'] > 0) | (emis['NOx'] > 0) | (emis['NH3'] > 0) | 
            (emis['SOx'] > 0) | (emis['PM2_5'] > 0)]

print(f"Filtered to {len(emis)} emission points with non-zero emissions")

# ================================
# Filter for power plants
# ================================

# This is already EGU CEMS data, so all should be power plants
# But check NAICS codes if available for validation
egu_naics_prefixes = ['2211', '221111', '221112', '221113', '221114', '221115', 
                      '221116', '221117', '221118', '221121', '221122']

# Create a mask for power plants
is_power_plant = emis['naics_code'].apply(
    lambda x: any(str(x).startswith(prefix) for prefix in egu_naics_prefixes) 
              if x else False
)

# Apply the mask to filter for power plants if NAICS codes are available and identified
if is_power_plant.any():
    egu_emis = emis[is_power_plant].copy()
    print(f"Filtered to {len(egu_emis)} power plant emission points using NAICS codes")
else:
    # Since this is EGU CEMS data, we can use all points even without NAICS codes
    print("Using all emission points (dataset is already for power plants - EGU CEMS)")
    egu_emis = emis.copy()

# ================================
# Inspect and validate the emissions data
# ================================

# Display summary statistics - properly handle the geometry column
print("\nEmissions Summary (in metric tonnes/year):")
# Convert to DataFrame to exclude geometry column for sum operation
emissions_df = pd.DataFrame(egu_emis.drop(columns=['geometry']))
emission_sums = emissions_df[["VOC", "NOx", "NH3", "SOx", "PM2_5", "CO2"]].sum()
print(emission_sums)

print("\nStack Parameter Statistics:")
stack_stats = emissions_df[["height", "diam", "temp", "velocity"]].describe()
print(stack_stats)

# Print the distribution of ORIS facility codes
print("\nNumber of unique ORIS facility codes:", egu_emis['oris_facility_code'].nunique())
print("Top 10 ORIS facilities by number of emission points:")
print(egu_emis['oris_facility_code'].value_counts().head(10))

# ================================
# Prepare Final Output for InMAP
# ================================

# Create a copy with just the InMAP required columns
inmap_columns = ["VOC", "NOx", "NH3", "SOx", "PM2_5", "height", "diam", "temp", "velocity", "geometry"]
inmap_egu = egu_emis[inmap_columns].copy()

# Save both versions - one with CO2 for climate modeling, one for InMAP
full_output_file = f"{output_dir}/processed_emissions_with_co2.gpkg"
egu_emis.to_file(full_output_file, driver="GPKG")
print(f"\nSaved complete emissions data (with CO2) to {full_output_file}")

inmap_output_file = f"{output_dir}/processed_emissions_for_inmap.gpkg"
inmap_egu.to_file(inmap_output_file, driver="GPKG")
print(f"Saved InMAP-formatted emissions data to {inmap_output_file}")

print("\nReady to use with run_sr function!")
print("Example: resultsISRM = run_sr(inmap_egu, model='isrm', emis_units='tons/year')")

# The emissions GeoDataFrame is now in the exact format needed for InMAP
egu_gdf = inmap_egu  # For use with run_sr
egu_with_co2 = egu_emis  # For use with climate damage calculations

Reading data from ../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv...
Data loaded. Shape: (126465, 77)
Loading CEMS facilities data...
Loaded 1343 unique CEMS facility IDs
Processed 2054 unique emission locations that match CEMS facility IDs
Filtered to 2050 emission points with non-zero emissions
Filtered to 2004 power plant emission points using NAICS codes

Emissions Summary (in metric tonnes/year):
VOC      1.837821e+04
NOx      6.414817e+05
NH3      1.244030e+04
SOx      7.354128e+05
PM2_5    9.662813e+04
CO2      2.389864e+07
dtype: float64

Stack Parameter Statistics:
            height         diam         temp     velocity
count  2004.000000  2004.000000  2004.000000  2004.000000
mean     52.914968     4.762724   526.243477    24.359109
std      53.650849     2.069812   207.497173    15.119923
min       0.000000     0.000000     0.000000     0.000000
25%      18.288000     3.352800   366.483333    16.459200
50%     

In [ ]:
# Below are the helper functions for the run_sr function
import time
import numpy as np
import zarr
from shapely.geometry import Polygon, Point
import pandas as pd
import geopandas as gpd
import s3fs

def rect(i, w, s, e, n):
    x = [w[i], e[i], e[i], w[i], w[i]]
    y = [s[i], s[i], n[i], n[i], s[i]]
    return x, y

def poly(sr):
    ret = []
    w = sr["W"][:]
    s = sr["S"][:]
    e = sr["E"][:]
    n = sr["N"][:]
    for i in range(52411):
        x, y = rect(i, w, s, e, n)
        ret.append(Polygon([[x[0],y[0]],[x[1],y[1]],[x[2],y[2]],
                            [x[3],y[3]],[x[4],y[4]]]))
    return ret

# Define the run_sr function
def run_sr(emis, model, emis_units="tons/year"):
    start = time.time()
    
    # Load spatial receptor grid (SR)
    url = 's3://inmap-model/isrm_v1.2.1.zarr/'
    fs = s3fs.S3FileSystem(anon=True, client_kwargs={"region_name": "us-east-2"})
    sr = zarr.open(
        store=url,
        mode="r",
        storage_options={"anon": True, "client_kwargs": {"region_name": "us-east-2"}}
    )   

    # Build the grid geometry
    p = poly(sr)
    print("Making polygons as geometry.")

    # Create grid GeoDataFrame
    df = pd.DataFrame({'Location': range(52411)})
    gdf = gpd.GeoDataFrame(df, geometry=p, crs="+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1")
    
    # Ensure emis has CRS set correctly
    if emis.crs is None:
        print("Warning: emis CRS is None. Assigning default CRS (WGS84).")
        emis = emis.set_crs("EPSG:4326")

    # Convert emissions to match grid CRS
    emis = emis.to_crs(gdf.crs)

    # Spatial join (match emissions to grid)
    join_right_df = gdf.sjoin(emis, how="right")

    # Debugging: Print missing locations
    missing_count = join_right_df.Location.isna().sum()
    print(f"Finished joining dataframes. Missing locations: {missing_count}")

    # Drop NaN locations if any exist
    join_right_df = join_right_df.dropna(subset=["Location"])
    
    index = join_right_df.Location.astype(int).tolist()  # Ensure integer type

    # Get unique indices for emissions
    ppl = np.unique(index).tolist()

    # Create dictionary for mapping locations to index
    dictionary = {ppl[i]: i for i in range(len(ppl))}

    # Load Source-Receptor (SR) matrix data
    SOA = sr['SOA'].get_orthogonal_selection(([0], ppl, slice(None)))
    print("SOA data allocated.")
    pNO3 = sr['pNO3'].get_orthogonal_selection(([0], ppl, slice(None)))
    print("pNO3 data allocated.")
    pNH4 = sr['pNH4'].get_orthogonal_selection(([0], ppl, slice(None)))
    print("pNH4 data allocated.")
    pSO4 = sr['pSO4'].get_orthogonal_selection(([0], ppl, slice(None)))
    print("pSO4 data allocated.")
    PM25 = sr['PrimaryPM25'].get_orthogonal_selection(([0], ppl, slice(None)))
    print("PrimaryPM25 data allocated.")

    # Initialize output data arrays
    SOA_data, pNO3_data, pNH4_data, pSO4_data, PM25_data = 0.0, 0.0, 0.0, 0.0, 0.0

    # Calculate pollution data using emissions
    for i in range(len(index)):
        loc_idx = dictionary[index[i]]  # Get correct index
        SOA_data += SOA[0, loc_idx, :] * emis.VOC.iloc[i]
        pNO3_data += pNO3[0, loc_idx, :] * emis.NOx.iloc[i]
        pNH4_data += pNH4[0, loc_idx, :] * emis.NH3.iloc[i]
        pSO4_data += pSO4[0, loc_idx, :] * emis.SOx.iloc[i]
        PM25_data += PM25[0, loc_idx, :] * emis.PM2_5.iloc[i]

    data = SOA_data + pNO3_data + pNH4_data + pSO4_data + PM25_data

    print("Accessing data.")

    # Apply emission unit conversion factor
    fact = 28766.639 if emis_units == "tons/year" else 1

    # Compute final pollution metrics
    TotalPM25 = fact * data
    TotalPop = sr['TotalPop'][0:52411]
    MortalityRate = sr['MortalityRate'][0:52411]
    deathsK = (np.exp(np.log(1.06)/10 * TotalPM25) - 1) * TotalPop * 1.04658 * MortalityRate / 100000 * 1.02523
    deathsL = (np.exp(np.log(1.14)/10 * TotalPM25) - 1) * TotalPop * 1.04658 * MortalityRate / 100000 * 1.02523

    # Create output GeoDataFrame
    ret = gpd.GeoDataFrame(pd.DataFrame({
        'SOA': fact * SOA_data,
        'pNO3': fact * pNO3_data,
        'pNH4': fact * pNH4_data,
        'pSO4': fact * pSO4_data,
        'PrimaryPM25': fact * PM25_data,
        'TotalPM25': TotalPM25,
        'deathsK': deathsK,
        'deathsL': deathsL
    }), geometry=p[:52411])

    print(f"Finished ({time.time() - start:.0f} seconds)")
    return ret

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.gridspec as gridspec
from pathlib import Path

# ================================
# Step 1: Load the necessary data
# ================================

# Load the emissions data with CO2 values
egu_with_co2 = gpd.read_file("../data/processed/processed_emissions_with_co2.gpkg")

# Load the InMAP-formatted emissions data for the SR model
inmap_egu = gpd.read_file("../data/processed/processed_emissions_for_inmap.gpkg")

# Run the Source-Receptor model with the InMAP-formatted data
print("Running Source-Receptor model...")
resultsISRM = run_sr(inmap_egu, model="isrm", emis_units="tons/year")

In [ ]:
# ================================
# Step 2: Prepare county-level data for visualization
# ================================

# Load county boundaries
print("Loading county boundaries...")
us_counties = gpd.read_file("../data/raw/cb_2018_us_county_500k/cb_2018_us_county_500k.shp")

# Convert counties to match emissions data CRS
us_counties = us_counties.to_crs(resultsISRM.crs)

# Perform spatial join to assign each grid cell to a county
print("Joining health impacts to counties...")
results_county = resultsISRM.sjoin(us_counties, how="left", predicate="intersects")

# Aggregate health impacts by county
county_summary = results_county.groupby("NAME").agg({
    "SOA": "sum",
    "pNO3": "sum",
    "pNH4": "sum",
    "pSO4": "sum",
    "PrimaryPM25": "sum",
    "TotalPM25": "sum",
    "deathsK": "sum",
    "deathsL": "sum"
}).reset_index()

# Merge summary with county shapefile for visualization
us_counties = us_counties.merge(county_summary, on="NAME", how="left")

# Exclude Alaska, Hawaii, and Puerto Rico
us_counties = us_counties[~us_counties['STATEFP'].isin(["02", "15", "72"])]

# ================================
# Step 3: Calculate health damages
# ================================

# Value of a Statistical Life in dollars
VSL = 13.2e6  # $13.2 million per life

# Calculate health damages for each county
us_counties['HealthDamages'] = us_counties['deathsK'] * VSL

# Handle NaN values
us_counties.loc[:, 'HealthDamages'] = us_counties['HealthDamages'].fillna(0)

# ================================
# Step 4: Calculate climate damages using actual CO2 emissions
# ================================

# Join the CO2 emissions to counties for climate damage calculation
print("Calculating climate damages from CO2 emissions...")

# Spatial join to assign CO2 emissions to counties
egu_with_co2 = egu_with_co2.to_crs(us_counties.crs)
co2_county = egu_with_co2.sjoin(us_counties[['GEOID', 'geometry']], how="inner", predicate="within")

# Aggregate CO2 emissions by county
co2_summary = co2_county.groupby('GEOID')['CO2'].sum().reset_index()

# Social Cost of Carbon (SCC) in dollars per ton
SCC = 51  # $51 per ton of CO2 (based on EPA estimates)

# Calculate climate damages for each county
co2_summary['ClimateDamages'] = co2_summary['CO2'] * SCC

# Merge climate damages with county data
us_counties = us_counties.merge(co2_summary[['GEOID', 'ClimateDamages']], on='GEOID', how='left')
us_counties['ClimateDamages'] = us_counties['ClimateDamages'].fillna(0)

# Calculate total damages (health + climate)
us_counties['TotalDamages'] = us_counties['HealthDamages'] + us_counties['ClimateDamages']

# ================================
# Step 5: Create dual-panel visualization
# ================================

# Define bins and colors for damages
bins = [0, 1e6, 5e6, 10e6, 50e6, 100e6, 500e6, 1e9, 5e9, float("inf")]
colors = ['#ffedea', '#ffcec5', '#ffad9f', '#ff7f66', '#ff4d33', 
          '#ff1a00', '#cc1600', '#990f00', '#660a00']

# Create a discrete colormap
cmap = mcolors.ListedColormap(colors)

# Bin the health and total damages data
us_counties['HealthDamages_Binned'] = pd.cut(us_counties['HealthDamages'], bins=bins, labels=False, include_lowest=True)
us_counties['TotalDamages_Binned'] = pd.cut(us_counties['TotalDamages'], bins=bins, labels=False, include_lowest=True)

# Create figure with two panels
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])

# Left panel - Health Damages
ax1 = plt.subplot(gs[0])
us_counties.plot(column='HealthDamages_Binned', cmap=cmap, linewidth=0.3, edgecolor="black", 
                 ax=ax1, legend=False)
ax1.set_title('Health Damages from Power Plant Emissions', fontsize=16)
ax1.axis('off')
ax1.set_aspect(1.3)
ax1.set_xlim(-130, -60)
ax1.set_ylim(20, 55)

# Calculate total damages across all counties
total_health = us_counties['HealthDamages'].sum() / 1e9  # billions of dollars
total_climate = us_counties['ClimateDamages'].sum() / 1e9  # billions of dollars
total_combined = us_counties['TotalDamages'].sum() / 1e9  # billions of dollars

# Add text annotation with total health damages
ax1.text(0.05, 0.05, f'Total Health Damages: ${total_health:.2f} billion', 
         transform=ax1.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

# Right panel - Total Damages (Health + Climate)
ax2 = plt.subplot(gs[1])
us_counties.plot(column='TotalDamages_Binned', cmap=cmap, linewidth=0.3, edgecolor="black", 
                 ax=ax2, legend=False)
ax2.set_title('Total Damages (Health + Climate) from Power Plant Emissions', fontsize=16)
ax2.axis('off')
ax2.set_aspect(1.3)
ax2.set_xlim(-130, -60)
ax2.set_ylim(20, 55)

# Add text annotation with total combined damages
ax2.text(0.05, 0.05, 
         f'Total Combined Damages: ${total_combined:.2f} billion\n' +
         f'Climate Damages: ${total_climate:.2f} billion\n' + 
         f'Using actual CO2 emissions data', 
         transform=ax2.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

# Create formatted legend labels
legend_labels = [
    '$0 - $1M', 
    '$1M - $5M', 
    '$5M - $10M', 
    '$10M - $50M', 
    '$50M - $100M', 
    '$100M - $500M', 
    '$500M - $1B', 
    '$1B - $5B', 
    '$5B+'
]

# Create custom legend patches
legend_patches = [
    mpatches.Patch(color=colors[i], label=legend_labels[i]) 
    for i in range(len(legend_labels))
]

# Add the custom legend to the figure
fig.legend(handles=legend_patches, title="Damages ($)", 
           loc="lower center", ncol=len(legend_labels), bbox_to_anchor=(0.5, 0.02))

# Adjust layout
plt.tight_layout(rect=[0, 0.07, 1, 0.98])
plt.suptitle('Comparison of Health vs. Combined (Health + Climate) Damages from Power Plant Emissions', fontsize=18, y=0.98)

# Create output directory if it doesn't exist
output_dir = Path("../figures")
output_dir.mkdir(parents=True, exist_ok=True)

# Save the figure
plt.savefig("../figures/health_climate_damages_comparison.png", dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

print("Visualization complete!")